<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [20]:
from fastcoref import FCoref

import pandas as pd
import tqdm

In [21]:
# get all the unique names and store them in a list
character_metadata = pd.read_csv('../data/character.metadata.tsv', sep='\t', header=None)
character_metadata.columns = [
    'wikipedia_movie_id',
    'freebase_movie_id',
    'movie_release_date',
    'character_name',
    'actor_dob',
    'actor_gender',
    'actor_height',
    'actor_ethnicity',
    'actor_name',
    'actor_age',
    'freebase_character_map_1',
    'freebase_character_map_2',
    'freebase_character_map_3'
]
characters = character_metadata['character_name'].unique().tolist()
characters = [name for name in characters if type(name) == str]
characters = ' '.join(characters)

In [22]:
# test text

text = """The nation of Panem consists of a wealthy Capitol and twelve poorer districts. As punishment for a past rebellion, each district must provide a boy and girl  between the ages of 12 and 18 selected by lottery  for the annual Hunger Games. The tributes must fight to the death in an arena; the sole survivor is rewarded with fame and wealth. In her first Reaping, 12-year-old Primrose Everdeen is chosen from District 12. Her older sister Katniss volunteers to take her place. Peeta Mellark, a baker's son who once gave Katniss bread when she was starving, is the other District 12 tribute. Katniss and Peeta are taken to the Capitol, accompanied by their frequently drunk mentor, past victor Haymitch Abernathy. He warns them about the "Career" tributes who train intensively at special academies and almost always win. During a TV interview with Caesar Flickerman, Peeta unexpectedly reveals his love for Katniss. She is outraged, believing it to be a ploy to gain audience support, as "sponsors" may provide in-Games gifts of food, medicine, and tools. However, she discovers Peeta meant what he said. The televised Games begin with half of the tributes killed in the first few minutes; Katniss barely survives ignoring Haymitch's advice to run away from the melee over the tempting supplies and weapons strewn in front of a structure called the Cornucopia. Peeta forms an uneasy alliance with the four Careers. They later find Katniss and corner her up a tree. Rue, hiding in a nearby tree, draws her attention to a poisonous tracker jacker nest hanging from a branch. Katniss drops it on her sleeping besiegers. They all scatter, except for Glimmer, who is killed by the insects. Hallucinating due to tracker jacker venom, Katniss is warned to run away by Peeta. Rue cares for Katniss for a couple of days until she recovers. Meanwhile, the alliance has gathered all the supplies into a pile. Katniss has Rue draw them off, then destroys the stockpile by setting off the mines planted around it. Furious, Cato kills the boy assigned to guard it. As Katniss runs from the scene, she hears Rue calling her name. She finds Rue trapped and releases her. Marvel, a tribute from District 1, throws a spear at Katniss, but she dodges the spear, causing it to stab Rue in the stomach instead. Katniss shoots him dead with an arrow. She then comforts the dying Rue with a song. Afterward, she gathers and arranges flowers around Rue's body. When this is televised, it sparks a riot in Rue's District 11. President Snow summons Seneca Crane, the Gamemaker, to express his displeasure at the way the Games are turning out. Since Katniss and Peeta have been presented to the public as "star-crossed lovers", Haymitch is able to convince Crane to make a rule change to avoid inciting further riots. It is announced that tributes from the same district can win as a pair. Upon hearing this, Katniss searches for Peeta and finds him with an infected sword wound in the leg. She portrays herself as deeply in love with him and gains a sponsor's gift of soup. An announcer proclaims a feast, where the thing each survivor needs most will be provided. Peeta begs her not to risk getting him medicine. Katniss promises not to go, but after he falls asleep, she heads to the feast. Clove ambushes her and pins her down. As Clove gloats, Thresh, the other District 11 tribute, kills Clove after overhearing her tormenting Katniss about killing Rue. He spares Katniss "just this time...for Rue". The medicine works, keeping Peeta mobile. Foxface, the girl from District 5, dies from eating nightlock berries she stole from Peeta; neither knew they are highly poisonous. Crane changes the time of day in the arena to late at night and unleashes a pack of hound-like creatures to speed things up. They kill Thresh and force Katniss and Peeta to flee to the roof of the Cornucopia, where they encounter Cato. After a battle, Katniss wounds Cato with an arrow and Peeta hurls him to the creatures below. Katniss shoots Cato to spare him a prolonged death. With Peeta and Katniss apparently victorious, the rule change allowing two winners is suddenly revoked. Peeta tells Katniss to shoot him. Instead, she gives him half of the nightlock. However, before they can commit suicide, they are hastily proclaimed the victors of the 74th Hunger Games. Haymitch warns Katniss that she has made powerful enemies after her display of defiance. She and Peeta return to District 12, while Crane is locked in a room with a bowl of nightlock berries, and President Snow considers the situation."""
text_split = text.split()
model = FCoref(device='cuda:0')

preds = model.predict(
    texts=text.split(),
    is_split_into_words=True
)
clusters = preds.get_clusters(as_strings=False)

# get the token id, start offset map
resolved_text = text.split()

for cluster in clusters:
    character_name = None
    for token_offset in cluster:
        token = ' '.join(text_split[token_offset[0]:token_offset[1]])
        if token in characters and token.lower() not in ["he", "him", "his", "she", "her", "hers", "they", "them", "their"]:
            character_name = token
        elif token.lower() in ["he", "him", "his", "she", "her", "hers", "they", "them", "their"] and character_name!=None:
            print(token, '->', character_name)
            resolved_text[token_offset[0]] = character_name

print(' '.join(resolved_text))

12/15/2023 14:20:35 - INFO - 	 missing_keys: []
12/15/2023 14:20:35 - INFO - 	 unexpected_keys: []
12/15/2023 14:20:35 - INFO - 	 mismatched_keys: []
12/15/2023 14:20:35 - INFO - 	 error_msgs: []
12/15/2023 14:20:35 - INFO - 	 Model Parameters: 90.5M, Transformer: 82.1M, Coref head: 8.4M
12/15/2023 14:20:35 - INFO - 	 Tokenize 1 inputs...
Map: 100%|██████████| 1/1 [00:00<00:00, 54.33 examples/s]
12/15/2023 14:20:36 - INFO - 	 ***** Running Inference on 1 texts *****
Inference: 100%|██████████| 1/1 [00:00<00:00,  3.84it/s]

she -> Katniss
She -> Katniss
she -> Katniss
her -> Katniss
her -> Katniss
she -> Katniss
she -> Katniss
her -> Katniss
She -> Katniss
she -> Katniss
She -> Katniss
she -> Katniss
She -> Katniss
her -> Katniss
she -> Katniss
her -> Katniss
her -> Katniss
her -> Katniss
she -> Katniss
her -> Katniss
She -> Katniss
his -> Peeta
he -> Peeta
him -> Peeta
him -> Peeta
him -> Peeta
he -> Peeta
she -> Peeta
his -> President Snow
him -> Cato
him -> Cato
him -> Cato
The nation of Panem consists of a wealthy Capitol and twelve poorer districts. As punishment for a past rebellion, each district must provide a boy and girl between the ages of 12 and 18 selected by lottery for the annual Hunger Games. The tributes must fight to the death in an arena; the sole survivor is rewarded with fame and wealth. In her first Reaping, 12-year-old Primrose Everdeen is chosen from District 12. Her older sister Katniss volunteers to take her place. Peeta Mellark, a baker's son who once gave Katniss bread when Kat

In [23]:
PLOT_SUMMARY_PATH = "../data/plot_summaries.txt"

plot_summaries = pd.read_csv(PLOT_SUMMARY_PATH, sep='\t', header=None)
plot_summaries.columns = ['wiki_id', 'plot']
plot_summaries

,wiki_id,plot
0,23890098,"Shlykov, a hard-working taxi driver and Lyosha..."
1,31186339,The nation of Panem consists of a wealthy Capi...
2,20663735,Poovalli Induchoodan is sentenced for six yea...
3,2231378,"The Lemon Drop Kid , a New York City swindler,..."
4,595909,Seventh-day Adventist Church pastor Michael Ch...
...,...,...
42298,34808485,"The story is about Reema , a young Muslim scho..."
42299,1096473,"In 1928 Hollywood, director Leo Andreyev look..."
42300,35102018,American Luthier focuses on Randy Parsons’ tra...
42301,8628195,"Abdur Rehman Khan , a middle-aged dry fruit se..."


In [25]:
resolved_texts = []

for idx, row in plot_summaries.iterrows():
    text = row['plot']
    text_split = text.split()
    resolved_text = text.split()
    preds = model.predict(
        texts=text.split(),
        is_split_into_words=True
    )
    clusters = preds.get_clusters(as_strings=False)
    for cluster in clusters:
        character_name = None
        for token_offset in cluster:
            token = ' '.join(text_split[token_offset[0]:token_offset[1]])
            if token in characters and token.lower() not in ["he", "him", "his", "she", "her", "hers", "they", "them", "their"]:
                character_name = token
            elif token.lower() in ["he", "him", "his", "she", "her", "hers", "they", "them", "their"] and character_name!=None:
                resolved_text[token_offset[0]] = character_name
    resolved_texts.append(' '.join(resolved_text))

wiki_ids = plot_summaries['wiki_id'].tolist()
resolved_df = pd.DataFrame({'wiki_id': wiki_ids, 'resolved_text': resolved_texts})
resolved_df.to_csv('../data/resolved_texts_fastcoref.csv', index=False)

# plot_summaries = plot_sum
# display(plot_summaries.head())
# plot_summaries.to_csv('../data/plot_summaries_resolved_fastcoref.csv', index=False)

12/15/2023 14:21:23 - INFO - 	 Tokenize 1 inputs...
Map: 100%|██████████| 1/1 [00:00<00:00, 63.68 examples/s]
12/15/2023 14:21:23 - INFO - 	 ***** Running Inference on 1 texts *****
Inference: 100%|██████████| 1/1 [00:00<00:00,  1.10it/s]
12/15/2023 14:21:24 - INFO - 	 Tokenize 1 inputs...
Map: 100%|██████████| 1/1 [00:00<00:00, 43.63 examples/s]
12/15/2023 14:21:24 - INFO - 	 ***** Running Inference on 1 texts *****
Inference: 100%|██████████| 1/1 [00:00<00:00,  3.33it/s]
12/15/2023 14:21:25 - INFO - 	 Tokenize 1 inputs...


she -> Katniss
She -> Katniss
she -> Katniss
her -> Katniss
her -> Katniss
she -> Katniss
she -> Katniss
her -> Katniss
She -> Katniss
she -> Katniss
She -> Katniss
she -> Katniss
She -> Katniss
her -> Katniss
she -> Katniss
her -> Katniss
her -> Katniss
her -> Katniss
she -> Katniss
her -> Katniss
She -> Katniss
his -> Peeta
he -> Peeta
him -> Peeta
him -> Peeta
him -> Peeta
he -> Peeta
she -> Peeta
his -> President Snow
him -> Cato
him -> Cato
him -> Cato


Map: 100%|██████████| 1/1 [00:00<00:00, 111.12 examples/s]
12/15/2023 14:21:25 - INFO - 	 ***** Running Inference on 1 texts *****
Inference: 100%|██████████| 1/1 [00:00<00:00, 11.94it/s]
12/15/2023 14:21:25 - INFO - 	 Tokenize 1 inputs...


his -> Induchoodan
he -> Induchoodan
his -> Induchoodan
his -> Induchoodan
his -> Induchoodan
his -> Induchoodan
his -> Induchoodan
his -> Induchoodan
his -> Menon
he -> Pavithran
his -> Pavithran


Map: 100%|██████████| 1/1 [00:00<00:00, 56.78 examples/s]
12/15/2023 14:21:25 - INFO - 	 ***** Running Inference on 1 texts *****
Inference: 100%|██████████| 1/1 [00:00<00:00,  8.56it/s]
12/15/2023 14:21:25 - INFO - 	 Tokenize 1 inputs...


...

his -> Jaya
his -> Jaya
He -> Jaya
his -> Jaya
her -> Noor
she -> Noor
her -> Noor
she -> Noor
her -> Noor
his -> Madakari Nayaka


Map: 100%|██████████| 1/1 [00:00<00:00, 62.24 examples/s]
12/15/2023 14:22:33 - INFO - 	 ***** Running Inference on 1 texts *****
Inference: 100%|██████████| 1/1 [00:00<00:00, 10.06it/s]
12/15/2023 14:22:33 - INFO - 	 Tokenize 1 inputs...


him -> Shawn
he -> Shawn
He -> Shawn
he -> Shawn
him -> Shawn
him -> Shawn
her -> Maddy
she -> Maddy
her -> Maddy
she -> Maddy
her -> Maddy
her -> Maddy
she -> Maddy
she -> Maddy
she -> Maddy
her -> Maddy
she -> Maddy
her -> Maddy
her -> Maddy
Her -> Maddy
she -> Maddy's
She -> Maddy
she -> Maddy
her -> Maddy
her -> Maddy
her -> Maddy
She -> Maddy
She -> Maddy
She -> Maddy
her -> Maddy
she -> Maddy
She -> Maddy
he -> Chris
he -> Chris
He -> Chris
his -> Chris
he -> Chris
him -> Chris
He -> Chris
he -> Chris
He -> Chris
He -> Chris
she -> Tillie
she -> Tillie
her -> Tillie
She -> Tillie
she -> Tillie
she -> Tillie
she -> Tillie
She -> Tillie
She -> Tillie


Map: 100%|██████████| 1/1 [00:00<00:00, 201.62 examples/s]
12/15/2023 14:22:33 - INFO - 	 ***** Running Inference on 1 texts *****
Inference: 100%|██████████| 1/1 [00:00<00:00, 19.66it/s]
12/15/2023 14:22:33 - INFO - 	 Tokenize 1 inputs...


KeyboardInterrupt: 